In [1]:
import pandas as pd

file_path = 'data.csv'

data = pd.read_csv(file_path)


data.head()


/tmp/ipykernel_6396/4144638285.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.result,verification.time
0,0,0,2,1,59,1,0,False,163.316667
1,0,0,2,1,59,2,0,False,200.860000
2,0,0,2,1,59,4,0,False,154.888889
3,0,0,2,1,59,6,0,False,108.640000
4,0,0,2,1,60,1,0,True,85.466667


In [2]:
missing_values = data.isnull().sum()
data_types = data.dtypes
missing_values, data_types, data.head()

(process.b1.capacity    0
 process.b2.capacity    0
 process.b3.capacity    0
 process.b4.capacity    0
 property.price         0
 property.product       0
 property.winner        0
 verification.result    0
 verification.time      0
 dtype: int64,
 process.b1.capacity      int64
 process.b2.capacity      int64
 process.b3.capacity      int64
 process.b4.capacity      int64
 property.price           int64
 property.product         int64
 property.winner          int64
 verification.result       bool
 verification.time      float64
 dtype: object,
    process.b1.capacity  process.b2.capacity  process.b3.capacity  \
 0                    0                    0                    2   
 1                    0                    0                    2   
 2                    0                    0                    2   
 3                    0                    0                    2   
 4                    0                    0                    2   
 
    process.b4.capacity  proper

In [3]:
# Фіча інжиніринг

# Створення нової фічі - сумарна ємність усіх процесів
data['total_capacity'] = data[['process.b1.capacity', 'process.b2.capacity', 'process.b3.capacity', 'process.b4.capacity']].sum(axis=1)

# Створення ще однієї фічі - відношення ціни до сумарної ємності
# Додаємо невелике число до знаменника, щоб уникнути ділення на нуль
data['price_to_capacity_ratio'] = data['property.price'] / (data['total_capacity'] + 0.0001)

# Переглянемо датасет з новими фічами
data.head()


,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.result,verification.time,total_capacity,price_to_capacity_ratio
0,0,0,2,1,59,1,0,False,163.316667,3,19.666011
1,0,0,2,1,59,2,0,False,200.860000,3,19.666011
2,0,0,2,1,59,4,0,False,154.888889,3,19.666011
3,0,0,2,1,59,6,0,False,108.640000,3,19.666011
4,0,0,2,1,60,1,0,True,85.466667,3,19.999333


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Вибираємо ознаки для масштабування
features_to_scale = data.columns.difference(['verification.result'])

# Ініціалізація масштабувальника
scaler = StandardScaler()

# Масштабування ознак
data[features_to_scale] = scaler.fit_transform(data[features_to_scale])

# Розділення даних на тренувальний, валідаційний та тестовий набори
X = data.drop('verification.result', axis=1)
y = data['verification.result']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Перевірка розмірів наборів даних
X_train.shape, X_valid.shape, X_test.shape


((1225, 10), (409, 10), (409, 10))

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Тренування базової моделі (логістична регресія)
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Оцінка точності моделі на валідаційному наборі
y_pred_valid = log_reg.predict(X_valid)
base_model_accuracy = accuracy_score(y_valid, y_pred_valid)
base_model_accuracy


0.902200488997555

In [6]:
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

# Визначення базових моделей для стекінгу
base_models = [
    ('knn', KNeighborsClassifier()),
    ('dt', DecisionTreeClassifier()),
    ('svc', SVC(probability=True))
]

# Ініціалізація стекінгу з логістичною регресією як мета-моделлю
stacking_classifier = StackingClassifier(estimators=base_models, final_estimator=LogisticRegression())

# Тренування моделі стекінгу
stacking_classifier.fit(X_train, y_train)

# Оцінка точності моделі стекінгу на валідаційному наборі
y_pred_valid_stacking = stacking_classifier.predict(X_valid)
stacking_model_accuracy = accuracy_score(y_valid, y_pred_valid_stacking)
stacking_model_accuracy


0.9926650366748166

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

# Тренування моделі бустінгу (Gradient Boosting Classifier)
gb_classifier = GradientBoostingClassifier()
gb_classifier.fit(X_train, y_train)

# Оцінка точності моделі бустінгу на валідаційному наборі
y_pred_valid_gb = gb_classifier.predict(X_valid)
boosting_model_accuracy = accuracy_score(y_valid, y_pred_valid_gb)
boosting_model_accuracy


0.9951100244498777

In [19]:
"""
Базова модель (логістична регресія): 90.22%
Стекінг: 99.27%
Бустінг (Gradient Boosting Classifier): 99.51%
Беггінг (Bagging Classifier): 99.51%
"""

from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score

# Тренування моделі беггінгу без явного вказівки base_estimator
bagging_classifier = BaggingClassifier(n_estimators=100, random_state=42)
bagging_classifier.fit(X_train, y_train)

# Оцінка точності моделі беггінгу на валідаційному наборі
y_pred_valid_bagging = bagging_classifier.predict(X_valid)
bagging_model_accuracy = accuracy_score(y_valid, y_pred_valid_bagging)
bagging_model_accuracy

0.9951100244498777